In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
!ls /data2/yt8m/code/yt8m

notebooks  sigmoid  softmax


In [3]:
# p = np.load('/data2/yt8m/code/yt8m/probs_rnn3_tr_0919_1943.npy', allow_pickle=True) lb0.5xx

In [17]:
p1 = np.load('/data2/yt8m/code/yt8m/softmax/rnn6_finetune_val_0921_5574.npy', allow_pickle=True)

In [16]:
p2 = np.load('/data2/yt8m/code/yt8m/softmax/rnn3_finetune_val_0919_5319.npy', allow_pickle=True)
p1[:2]

array([[9.0666568e-01, 2.5496540e-06, 1.0090392e-06, ..., 2.5658448e-05,
        6.0842640e-06, 2.5115046e-06],
       [9.2794991e-01, 6.5408211e-05, 6.3841624e-05, ..., 5.5461137e-06,
        2.5619098e-04, 7.3228948e-06]], dtype=float32)

In [18]:
p = np.average([p1, p2], 0, weights=[0.5, 0.5])

In [19]:
type(p)

numpy.ndarray

In [20]:
p[:2]

array([[8.37563097e-01, 7.60683963e-06, 4.16047004e-06, ...,
        5.82569237e-05, 1.08952231e-05, 2.87129335e-05],
       [8.88292313e-01, 4.95916356e-05, 6.87898973e-05, ...,
        7.85053703e-06, 8.34359540e-04, 2.32790499e-05]])

In [21]:
df_test = pd.read_csv('/data2/yt8m/3/frame/test_ids.csv')
df_test.head()

,vid,start_time,seg_id
0,MeVj,0,MeVj:0
1,MeVj,5,MeVj:5
2,MeVj,10,MeVj:10
3,MeVj,15,MeVj:15
4,MeVj,20,MeVj:20


In [22]:
def get_classes():
    df = pd.read_csv('/data2/yt8m/3/frame/classes_1001.csv')
    c = df['class'].values.tolist()
    stoi = { c[i]: i for i in range(len(c)) }
    return c, stoi

classes, _ = get_classes()

In [23]:
classes[:2]

['none', '3']

In [24]:
print(df_test.shape, p.shape)

(2038114, 3) (2038114, 1001)


In [25]:
res = []
for i in tqdm(range(1, 1001)):
    idx = np.argsort(p[:,i])[::-1][:100000]
    best_segs = df_test.seg_id.values[idx]
    pred_str = ' '.join(best_segs)
    res.append({
        'Class': classes[i],
        'Segments': pred_str
    })


100%|██████████| 1000/1000 [07:29<00:00,  2.23it/s]


In [26]:
df_sub = pd.DataFrame(res)
df_sub.head()

,Class,Segments
0,3,1shy:15 1shy:150 1shy:95 1shy:100 F4ZL:100 F4Z...
1,7,O7Z1:235 O7Z1:95 O7Z1:90 0UPH:175 O7Z1:240 O7Z...
2,8,zRMu:60 hcQU:255 Xhnt:40 sDmH:145 rUf6:110 qnS...
3,11,j6wU:220 DKzl:190 0F9k:125 bQG6:255 9VEE:110 2...
4,12,FXxf:100 L7U7:15 JJ9a:210 FXxf:70 FXxf:110 odz...


In [27]:
df_sub.to_csv('ens_finetune_5574_5319.csv', index=False)

In [28]:
!kaggle competitions submit -c youtube8m-2019 -f ens_finetune_5574_5319.csv -m 'submit'

100%|████████████████████████████████████████| 812M/812M [00:47<00:00, 17.9MB/s]
Successfully submitted to The 3rd YouTube-8M Video Understanding Challenge

In [13]:
!ls -lh

total 6.5G
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users 6.0M Sep 19 22:39 create_meta.ipynb
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users  41K Sep 18 15:56 eda.ipynb
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users 812M Sep 19 06:41 ens_0919_1.csv
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users 811M Sep 21 05:41 rnn6_softmax_finetune_5574.csv
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users 809M Sep 19 20:12 sigmoid_1.csv
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users 810M Sep 17 19:27 sub1_rnn3_local057_100000.csv
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users  82M Sep 17 19:16 sub1_rnn3_local057_10000.csv
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users  17M Sep 17 19:06 sub1_rnn3_local057_2000.csv
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users 8.2M Sep 17 03:34 sub1_rnn3_local0.57.csv
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users 810M Sep 18 04:53 sub_ens1.csv
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users  13K Sep 21 05:39 submit.ipynb
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users  1

In [19]:
del res

In [20]:
del p